대회 제출 형식으로 변환

In [7]:
# save csv

import os
import torch
import random
import numpy as np

import re
import glob
import pandas as pd
from pathlib import Path

def xywh2xyxy(x, origin_img_size):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.copy()
    y[0] = x[0] - x[2] / 2.  # top left x
    y[1] = x[1] - x[3] / 2.  # top left y
    y[2] = x[0] + x[2] / 2.  # bottom right x
    y[3] = x[1] + x[3] / 2.  # bottom right y
    y = np.array(y) * origin_img_size
    y = np.clip(y, 0, origin_img_size)
    return y

### 이건 train, detect 과정 상관없이 원본 이미지 사이즈인 1024로 고정인 듯??
origin_img_size = 1024

result_path = Path('./runs/detect/exp19').resolve()
# result_path = Path('./runs/val/exp9').resolve()

result_label = list(result_path.glob('labels/*.txt'))

result_label = sorted(result_label, key=lambda x: str(x).split('/')[-1].split('.'[0]))

prediction_strings = []
file_names = []
for image in result_label:
    with open(str(image),'r') as f:
        img_name = image.name.replace('.txt', '.jpg')
        img_name = 'test/' + img_name
        
        bbox_lines = f.readlines()
        bbox_lines = [line[:-1].split(' ') for line in bbox_lines]
        
        bbox_list = np.array(bbox_lines).reshape(-1,6)
        sort_list = sorted(bbox_list, key=lambda x: ( int(x[0]), -float(x[-1]) ) )

        prediction_string = ''
        for line in sort_list:
            classes, xc, yc, w, h, conf = line
            xc,yc,w,h,conf = list(map(float,[xc,yc,w,h,conf]))
            
            x1,y1,x2,y2 = xywh2xyxy([xc,yc,w,h], origin_img_size)
            x1,y1,x2,y2,conf = list(map(str,[x1,y1,x2,y2,conf]))

            prediction_string += f'{classes} {conf} {x1} {y1} {x2} {y2} '
            
        prediction_strings.append(prediction_string)
        file_names.append(img_name)

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(f'./fold_5_yolo.csv', index=None)
submission.head()

,PredictionString,image_id
0,0 0.258545 216.000256 689.999616 256.000256000...,test/0000.jpg
1,0 0.251221 743.000064 655.000064 895.000576 98...,test/0001.jpg
2,0 0.732422 160.9996288 287.99948800000004 241....,test/0002.jpg
3,9 0.933594 144.0 262.00064000000003 908.000256...,test/0003.jpg
4,0 0.750977 190.00012799999996 252.000255999999...,test/0004.jpg
